In [ ]:
!pip install beautifulsoup4 lxml
!pip install fastparquet
!Drd4pip install pyarrow
!pip install geopandas

In [4]:
import requests
from bs4 import BeautifulSoup
import re
import os
from pathlib import Path
import pandas as pd
import glob
from statistics import NormalDist
import geopandas as gpd
import zipfile

In [6]:
def file_detect(file, folder_path):
    if os.path.abspath(file) != file:
        file = folder_path + file
    folder_path = os.path.abspath(folder_path)
    return os.path.commonpath([file, folder_path]) == folder_path and os.path.isfile(file)


def sample_calc(population, confi_inter, p, error):
    n = population
    z = NormalDist().inv_cdf((1 + confi_inter) / 2.)
    q = 1 - p
    e = error
    n_0 = (z ** 2 * p * q) / (e ** 2)
    n_final = n_0 / (1 + (n_0 - 1) / n)
    return int(n_final)

In [8]:
url = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
directory_path = "/Users/olaf/Columbia_Source/IEOR_4501/final_project/"
directory = 'data/'

In [10]:
data_path = os.path.join(directory_path, directory)

os.makedirs(data_path, exist_ok=True)
data_path

'/Users/olaf/Columbia_Source/IEOR_4501/final_project/data/'

In [12]:

response = requests.get(url)
if response.status_code != 200:
    print("request failed, status code:", response.status_code)
    exit()

In [14]:

soup = BeautifulSoup(response.text, 'lxml')


Yellow_Taxi_pattern = re.compile(r'yellow_tripdata_202[0-4]-\d{2}.parquet', re.IGNORECASE)
HVFHV_pattern = re.compile(r'fhvhv_tripdata_202[0-4]-\d{2}.parquet', re.IGNORECASE)

Yellow_Taxi_links = [link.get('href') for link in soup.find_all('a', href=Yellow_Taxi_pattern)]
HVFHV_links = [link.get('href') for link in soup.find_all('a', href=HVFHV_pattern)]

all_links = Yellow_Taxi_links + HVFHV_links

# Download the file if it is not in the folder 
for link in all_links:

    
    file_url = link if link.startswith('http') else url + link
    file_name = file_url.split('/')[-1]
    abs_file_name = os.path.join(data_path, file_name)
    if file_detect(file_name, data_path) == False:    
        print(f"Downloading {file_name} ...")
        file_response = requests.get(file_url)
        
        with open(abs_file_name, 'wb') as file:
            file.write(file_response.content)
    else:
        print(f"{file_name} has been downloaded！")

yellow_tripdata_2024-01.parquet  has been downloaded！
yellow_tripdata_2024-02.parquet  has been downloaded！
yellow_tripdata_2024-03.parquet  has been downloaded！
yellow_tripdata_2024-04.parquet has been downloaded！
yellow_tripdata_2024-05.parquet has been downloaded！
yellow_tripdata_2024-06.parquet has been downloaded！
yellow_tripdata_2024-07.parquet has been downloaded！
yellow_tripdata_2024-08.parquet has been downloaded！
yellow_tripdata_2024-09.parquet has been downloaded！
yellow_tripdata_2023-01.parquet has been downloaded！
yellow_tripdata_2023-02.parquet has been downloaded！
yellow_tripdata_2023-03.parquet has been downloaded！
yellow_tripdata_2023-04.parquet has been downloaded！
yellow_tripdata_2023-05.parquet  has been downloaded！
yellow_tripdata_2023-06.parquet has been downloaded！
yellow_tripdata_2023-07.parquet  has been downloaded！
yellow_tripdata_2023-08.parquet  has been downloaded！
yellow_tripdata_2023-09.parquet  has been downloaded！
yellow_tripdata_2023-10.parquet  has be

In [16]:
df = pd.read_parquet(rf'{data_path}yellow_tripdata_2023-12.parquet', engine='pyarrow')
yellow_sample = pd.DataFrame(columns=df.columns)

print(df.head())


   VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0         1  2023-12-01 00:06:06   2023-12-01 00:15:47              0.0   
1         1  2023-12-01 00:22:26   2023-12-01 00:28:53              0.0   
2         1  2023-12-01 00:59:44   2023-12-01 01:13:22              2.0   
3         2  2023-12-01 00:22:17   2023-12-01 00:30:59              1.0   
4         2  2023-12-01 00:18:16   2023-12-01 00:25:32              2.0   

   trip_distance  RatecodeID store_and_fwd_flag  PULocationID  DOLocationID  \
0           1.10         1.0                  N           230            48   
1           1.50         1.0                  N           142           238   
2           2.20         1.0                  N           114           186   
3           0.66         1.0                  N            79            79   
4           2.20         1.0                  N           229           263   

   payment_type  fare_amount  extra  mta_tax  tip_amount  tolls_amount  \


In [18]:
df = pd.read_parquet(rf'{data_path}fhvhv_tripdata_2020-12.parquet', engine='pyarrow')
uber_sample = pd.DataFrame(columns=df.columns)
print(df.head())


  hvfhs_license_num dispatching_base_num originating_base_num  \
0            HV0003               B02764               B02764   
1            HV0003               B02764               B02764   
2            HV0005               B02510                 None   
3            HV0003               B02883               B02883   
4            HV0003               B02883               B02883   

     request_datetime   on_scene_datetime     pickup_datetime  \
0 2020-12-01 00:08:29 2020-12-01 00:11:29 2020-12-01 00:13:22   
1 2020-12-01 00:44:34 2020-12-01 00:47:00 2020-12-01 00:47:19   
2 2020-12-01 00:10:54                 NaT 2020-12-01 00:17:14   
3 2020-11-30 23:58:37 2020-12-01 00:00:01 2020-12-01 00:01:16   
4 2020-12-01 00:25:44 2020-12-01 00:30:41 2020-12-01 00:32:03   

     dropoff_datetime  PULocationID  DOLocationID  trip_miles  ...  sales_tax  \
0 2020-12-01 00:33:53            94            75        6.90  ...       1.79   
1 2020-12-01 00:57:01            75           164       

In [20]:
yellow_paths = glob.glob(rf'{data_path}yellow_tripdata_202[0-4]-*.parquet')
fhvhv_paths = glob.glob(rf'{data_path}fhvhv_tripdata_202[0-4]-*.parquet')
uber_identifiers = ['HV0003']

In [22]:
yellow_pop = 0
uber_pop = 0
for file in yellow_paths:
    df = pd.read_parquet(file, engine='pyarrow')
    yellow_pop += df[df.columns[0]].count()
    # try:
    #     yellow_data = df[['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 'PULocationID', 'DOLocationID']]
    #     yellow_data.rename(columns={'tpep_pickup_datetime': 'pickup_time', 'tpep_dropoff_datetime': 'dropoff_time', 'PULocationID': 'pickup_location_id', 'DOLocationID': 'dropoff_location_id'},inplace=True)
    #     yellow_data.to_parquet(file_path, index=False, engine='pyarrow')
    # except KeyError as e:
    #     print('The column names have been changed')
for file in fhvhv_paths:
    df = pd.read_parquet(file, engine='pyarrow')
    df = df[df['hvfhs_license_num'].isin(uber_identifiers)]
    uber_pop += df[df.columns[0]].count()
yellow_pop


135032467

In [52]:
yellow_sample_size = sample_calc(yellow_pop, 0.95, 0.5, 0.05)
samples = []
if file_detect('yellow_sample.csv', directory_path) == False:      
    for file in yellow_paths:
        df = pd.read_parquet(file, engine='pyarrow')
        sample_data = df.sample(n=yellow_sample_size, random_state=42)
        samples.append(sample_data)
    yellow_sample = pd.concat(samples, ignore_index=True)
    try:
        yellow_sample = yellow_sample[['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 'PULocationID', 'DOLocationID']]
    except KeyError as e:
        print('The column names have been changed')
    yellow_sample.to_csv(f'{directory_path}yellow_sample.csv', index=False)
else:
    print('yellow_sample.csv has been created. Loading..')
    yellow_sample = pd.read_csv(f'{directory_path}yellow_sample.csv')

yellow_sample.csv has been created. Loading..


In [26]:
print(yellow_sample.head())

  tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  trip_distance  \
0  2023-06-28 08:27:09   2023-06-28 09:21:52              1.0          10.22   
1  2023-06-13 22:05:38   2023-06-13 22:10:48              1.0           0.80   
2  2023-06-09 10:25:49   2023-06-09 10:41:51              2.0           1.62   
3  2023-06-28 15:56:14   2023-06-28 17:22:03              1.0          18.90   
4  2023-06-22 07:12:42   2023-06-22 07:23:47              1.0           1.52   

   PULocationID  DOLocationID  
0           138           144  
1           263           237  
2           162           236  
3           229             1  
4            43            74  


In [28]:
uber_sample_size = sample_calc(uber_pop, 0.95, 0.5, 0.05)

samples = []

if file_detect('uber_sample.csv', directory_path) == False:      
    for file in fhvhv_paths:
        df = pd.read_parquet(file, engine='pyarrow')
        df = df[df['hvfhs_license_num'].isin(uber_identifiers)]
        sample_data = df.sample(n=uber_sample_size, random_state=42)
        samples.append(sample_data)
    uber_sample = pd.concat(samples, ignore_index=True)    
    try: 
        uber_sample = uber_sample[['hvfhs_license_num', 'pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID', 'trip_miles', 'trip_time']]
    except KeyError as e:
        print('The column names have been changed')
    uber_sample.to_csv(f'{directory_path}uber_sample.csv', index=False)
else:
    print('uber_sample.csv has been created. Loading..')
    uber_sample = pd.read_csv(f'{directory_path}uber_sample.csv')

uber_sample.csv has been created. Loading..


In [30]:
print(uber_sample.head())

  hvfhs_license_num      pickup_datetime     dropoff_datetime  PULocationID  \
0            HV0003  2021-03-19 08:11:50  2021-03-19 08:20:59            97   
1            HV0003  2021-03-01 22:08:28  2021-03-01 22:20:19           247   
2            HV0003  2021-03-13 05:48:40  2021-03-13 05:55:36            18   
3            HV0003  2021-03-12 21:02:02  2021-03-12 21:25:13           211   
4            HV0003  2021-03-31 08:49:53  2021-03-31 09:56:02            74   

   DOLocationID  trip_miles  trip_time  
0           231        2.52        549  
1            20        2.41        711  
2           247        2.21        416  
3           229        3.85       1391  
4            45       18.96       3969  


In [32]:
link = 'https://d37ci6vzurychx.cloudfront.net/misc/taxi_zones.zip'
zone_zipfile = 'taxi_zones.zip'
zone_zipfile_abs = data_path + zone_zipfile
zone_file = 'taxi_zones.shp'

if file_detect(zone_zipfile, data_path) == False:      
    response = requests.get(link)
    with open(zone_zipfile_abs, 'wb') as file:
        file.write(response.content)
if file_detect(zone_file, data_path) == False:
    with zipfile.ZipFile(zone_zipfile_abs, 'r') as zip_ref:
        zip_ref.extractall(data_path)

# 加载包含区域信息的 Shapefile 文件
zone_gdf = gpd.read_file(f"{data_path}taxi_zones.shp")

# 查看列名
print(zone_gdf.columns)


Index(['OBJECTID', 'Shape_Leng', 'Shape_Area', 'zone', 'LocationID', 'borough',
       'geometry'],
      dtype='object')


In [34]:

try:    
    
    zone_gdf['centroid'] = zone_gdf.geometry.centroid
    zone_gdf['centroid'] = zone_gdf['centroid'].to_crs(epsg=4326)
    zone_gdf['latitude'] = zone_gdf['centroid'].y
    zone_gdf['longitude'] = zone_gdf['centroid'].x
    
    zone_gdf = zone_gdf[['LocationID', 'latitude', 'longitude']]
except AttributeError as e:
    print('The DataFrame has been edited')

In [222]:
print(zone_gdf[zone_gdf.duplicated(subset=['LocationID'])])
print(zone_gdf['LocationID'].to_string())

     LocationID   latitude  longitude
56           56  40.751819 -73.853582
103         103  40.698769 -74.040771
104         103  40.688784 -74.019073
0        1
1        2
2        3
3        4
4        5
5        6
6        7
7        8
8        9
9       10
10      11
11      12
12      13
13      14
14      15
15      16
16      17
17      18
18      19
19      20
20      21
21      22
22      23
23      24
24      25
25      26
26      27
27      28
28      29
29      30
30      31
31      32
32      33
33      34
34      35
35      36
36      37
37      38
38      39
39      40
40      41
41      42
42      43
43      44
44      45
45      46
46      47
47      48
48      49
49      50
50      51
51      52
52      53
53      54
54      55
55      56
56      56
57      58
58      59
59      60
60      61
61      62
62      63
63      64
64      65
65      66
66      67
67      68
68      69
69      70
70      71
71      72
72      73
73      74
74      75
75      76
76      77
7

In [144]:
# Latitude and Longitude Range
lat_min, lon_min = 40.560445, -74.242330
lat_max, lon_max = 40.908524, -73.717047
    
# Match the latitude and longitude of the dataset
if 'LocationID_pickup' not in yellow_sample:
    yellow_sample = yellow_sample.merge(zone_gdf, how='left', left_on='PULocationID', right_on='LocationID', suffixes=('', '_pickup'))
if 'LocationID_dropoff' not in yellow_sample:
    yellow_sample = yellow_sample.merge(zone_gdf, how='left', left_on='DOLocationID', right_on='LocationID', suffixes=('', '_dropoff'))
 
# Keep the rows that has different pickup location and drop location, and both pick up and drop off location id are in the zone dataframe
yellow_sample = yellow_sample[(yellow_sample['PULocationID'] != yellow_sample['DOLocationID'])
& (yellow_sample['trip_distance'] > 0) 
& (yellow_sample['PULocationID'] <= 263) 
& (yellow_sample['DOLocationID'] <= 263) 
]

# Drop the duplicated columns
yellow_sample = yellow_sample.loc[:,~yellow_sample.columns.duplicated()]

# Change the dtype of both pick up and drop off location id to int64
yellow_sample['LocationID_pickup'] = yellow_sample['LocationID_pickup'].astype('int64')
yellow_sample['LocationID_dropoff'] = yellow_sample['LocationID_dropoff'].astype('int64')

# Sanity Check: All the PULocationID should match pick up location id, and DOLocationID should match drop off location id
if yellow_sample['PULocationID'].all() != yellow_sample['LocationID_pickup'].all() & yellow_sample['DOLocationID'].all() != yellow_sample['LocationID_dropoff'].all():
    raise('Pickup Location or Dropoff Location ID not matached. Something went wrong!')

# Rename the columns
yellow_sample.rename(columns={
    'tpep_pickup_datetime': 'PUDatetime',
    'tpep_dropoff_datetime': 'DODatetime'
}, inplace=True)

# Set the appropriate dtype for each column
yellow_sample['PUDatetime'] = pd.to_datetime(yellow_sample['PUDatetime'])
yellow_sample['DODatetime'] = pd.to_datetime(yellow_sample['DODatetime'])
yellow_sample['trip_distance'] = yellow_sample['trip_distance'].astype(float)

# Keep the data only within the range 
yellow_sample = yellow_sample[
    (yellow_sample['latitude_pickup'].between(lat_min, lat_max)) & 
    (yellow_sample['longitude_pickup'].between(lon_min, lon_max)) &
    (yellow_sample['latitude_dropoff'].between(lat_min, lat_max)) &
    (yellow_sample['longitude_dropoff'].between(lon_min, lon_max))
]

# Drop the columns useless columns
yellow_sample = yellow_sample.drop(['PULocationID', 'DOLocationID', 'LocationID', 'latitude', 'longitude'], axis=1)

yellow_sample


,PUDatetime,DODatetime,passenger_count,trip_distance,LocationID_dropoff,latitude_dropoff,longitude_dropoff,LocationID_pickup,latitude_pickup,longitude_pickup
0,2023-06-28 08:27:09,2023-06-28 09:21:52,1.0,10.22,144,40.720889,-73.996919,138,40.774376,-73.873628
1,2023-06-13 22:05:38,2023-06-13 22:10:48,1.0,0.80,237,40.768615,-73.965635,263,40.778766,-73.951010
2,2023-06-09 10:25:49,2023-06-09 10:41:51,2.0,1.62,236,40.780436,-73.957012,162,40.756688,-73.972356
3,2023-06-28 15:56:14,2023-06-28 17:22:03,1.0,18.90,1,40.691830,-74.174002,229,40.756729,-73.965146
4,2023-06-22 07:12:42,2023-06-22 07:23:47,1.0,1.52,74,40.801169,-73.937346,43,40.782477,-73.965555
...,...,...,...,...,...,...,...,...,...,...
17928,2023-03-31 17:46:57,2023-03-31 17:56:36,1.0,1.29,68,40.748427,-73.999918,164,40.748575,-73.985156
17929,2023-03-02 11:43:52,2023-03-02 11:54:43,0.0,1.40,114,40.728340,-73.997380,90,40.742279,-73.996971
17930,2023-03-12 22:04:08,2023-03-12 22:13:13,5.0,0.97,234,40.740337,-73.990458,90,40.742279,-73.996971
17931,2023-03-06 20:59:31,2023-03-06 21:05:50,2.0,0.90,162,40.756688,-73.972356,163,40.764421,-73.977569


In [166]:
# Latitude and Longitude Range
lat_min, lon_min = 40.560445, -74.242330
lat_max, lon_max = 40.908524, -73.717047
    
# Match the latitude and longitude of the dataset
if 'LocationID_pickup' not in uber_sample:
    uber_sample = uber_sample.merge(zone_gdf, how='left', left_on='PULocationID', right_on='LocationID', suffixes=('', '_pickup'))
if 'LocationID_dropoff' not in uber_sample:
    uber_sample = uber_sample.merge(zone_gdf, how='left', left_on='DOLocationID', right_on='LocationID', suffixes=('', '_dropoff'))


# Keep the rows that has different pickup location and drop location, and both pick up and drop off location id are in the zone dataframe
uber_sample = uber_sample[(uber_sample['PULocationID'] != uber_sample['DOLocationID']) & 
(uber_sample['trip_miles'] > 0) & 
(uber_sample['PULocationID'] <= 263) & 
(uber_sample['DOLocationID'] <= 263) &
(uber_sample[
 
]

# Drop the duplicated columns
uber_sample = uber_sample.loc[:,~uber_sample.columns.duplicated()]

# # Change the dtype of both pick up and drop off location id to int64
# uber_sample['LocationID_pickup'] = uber_sample['LocationID_pickup'].astype('int64')
# uber_sample['LocationID_dropoff'] = uber_sample['LocationID_dropoff'].astype('int64')

# # Sanity Check: All the PULocationID should match pick up location id, and DOLocationID should match drop off location id
# if uber_sample['PULocationID'].all() != uber_sample['LocationID_pickup'].all() & uber_sample['DOLocationID'].all() != uber_sample['LocationID_dropoff'].all():
#     raise('Pickup Location or Dropoff Location ID not matached. Something went wrong!')

# # Rename the columns
# uber_sample.rename(columns={
#     'tpep_pickup_datetime': 'PUDatetime',
#     'tpep_dropoff_datetime': 'DODatetime',
#     'trip_miles': 'trip_distance'
# }, inplace=True)

# # Set the appropriate dtype for each column
# uber_sample['PUDatetime'] = pd.to_datetime(yellow_sample['PUDatetime'])
# uber_sample['DODatetime'] = pd.to_datetime(yellow_sample['DODatetime'])
# uber_sample['trip_distance'] = uber_sample['trip_distance'].astype(float)

# # Keep the data only within the range 
# uber_sample = uber_sample[
#     (uber_sample['latitude_pickup'].between(lat_min, lat_max)) & 
#     (uber_sample['longitude_pickup'].between(lon_min, lon_max)) &
#     (uber_sample['latitude_dropoff'].between(lat_min, lat_max)) &
#     (uber_sample['longitude_dropoff'].between(lon_min, lon_max))
# ]

# # Drop the columns useless columns
# uber_sample = uber_sample.drop(['PULocationID', 'DOLocationID', 'LocationID', 'latitude', 'longitude'], axis=1)

# uber_sample


In [168]:
#print(uber_sample.head())
uber_sample[uber_sample['LocationID_pickup'].isna()]

,hvfhs_license_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,trip_time,LocationID,latitude,longitude,LocationID_dropoff,latitude_dropoff,longitude_dropoff,LocationID_pickup,latitude_pickup,longitude_pickup
3006,HV0003,2021-09-18 21:20:24,2021-09-18 21:37:18,57,130,4.88,1014,NaN,NaN,NaN,130.0,40.704369,-73.793981,NaN,NaN,NaN
9545,HV0003,2020-03-19 15:22:17,2020-03-19 15:31:08,57,82,2.24,531,NaN,NaN,NaN,82.0,40.739495,-73.877118,NaN,NaN,NaN
15627,HV0003,2024-07-01 11:02:12,2024-07-01 11:19:40,57,16,7.74,1048,NaN,NaN,NaN,16.0,40.762738,-73.773421,NaN,NaN,NaN
15715,HV0003,2024-07-21 13:19:07,2024-07-21 13:50:07,57,222,12.17,1860,NaN,NaN,NaN,222.0,40.647527,-73.882413,NaN,NaN,NaN


In [146]:
weather_df = pd.read_csv("daily_weather_data.csv")  # 加载每日天气数据

weather_df['date'] = pd.to_datetime(weather_df['date'])

weather_df.set_index('date', inplace=True)
hourly_weather_df = weather_df.resample('H').interpolate(method='linear')

hourly_weather_df.to_csv("hourly_weather_data.csv")


FileNotFoundError: [Errno 2] No such file or directory: 'daily_weather_data.csv'